# Example 2: build a network using torch

In [2]:
import onnx
import onnxruntime as ort
import numpy as np
from onnx import shape_inference
import os
import sys
import torch
import streamease as se

In [ ]:
from helpers.data_loader import load_mnist_data

# Example usage:
input_shape = (28, 32)

# Call the functions with the instance
train_data, test_data,noisy_train_data, noisy_test_data = load_mnist_data(input_shape[0], input_shape[1], transpose= True)

Create the model

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim

def CausalConv1d(in_channels, out_channels, kernel_size, dilation=1, **kwargs):
   pad = (kernel_size - 1) * dilation
   return nn.Conv1d(in_channels, out_channels, kernel_size, padding=pad, dilation=dilation, bias=False, **kwargs)

class Net(nn.Module):
    def __init__(self, input_size):
        super(Net, self).__init__()

        # Create the model layers
        self.conv1 = CausalConv1d(in_channels=input_size, out_channels=36, kernel_size=3, dilation=1)
        self.conv2 = CausalConv1d(in_channels=36, out_channels=40, kernel_size=5, dilation=2)
        self.conv3 = CausalConv1d(in_channels=40, out_channels=input_size, kernel_size=2, dilation=4)
    
    def forward(self, x):

        x = self.conv1(x)
        x = torch.relu(x)
        if self.conv1.padding[0] != 0:
            x = x[:, :, :-self.conv1.padding[0]]  # remove trailing padding

        x = torch.relu(self.conv2(x))
        if self.conv2.padding[0] != 0:
            x = x[:, :, :-self.conv2.padding[0]]  # remove trailing padding

        out = self.conv3(x)
        if self.conv3.padding[0] != 0:
            out = out[:, :, :-self.conv3.padding[0]]  # remove trailing padding

        return out



def train(model, train_loader, num_epochs=4):
    # Define the loss and optimizer
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.002)  
    for epoch in range(num_epochs):
        for data in train_loader:
            inputs = data[0].float()  # Assuming float data type
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, inputs)
            loss.backward()
            optimizer.step()
        
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')
    return model
def test(model, test_loader):
    model.eval()
    all_outputs = []

    with torch.no_grad():
        for data in test_loader:
            inputs = data[0].float()
            outputs = model(inputs)
            all_outputs.append(outputs)

    return torch.cat(all_outputs, dim=0)
def save_onnx(model, dummy_input, path='.', onnx_filename='torch_original_model'):
    full_path = os.path.join(path, f"{onnx_filename}.onnx")
    # Export the model to ONNX
    torch.onnx.export(model, (dummy_input,), full_path, verbose=False, input_names=['input'], output_names=['output'])

In [ ]:

# Create an instance of the class
torch_model = Net(input_shape[1]) 

print(torch_model)

data loader

In [6]:
from helpers.preprocessing import torch_data_loader
batch_size =128
train_loader = torch_data_loader( noisy_train_data, batch_size)
test_loader = torch_data_loader(noisy_test_data, batch_size)

train the network

In [ ]:
epochs = 4

torch_model = train(torch_model, train_loader, epochs)

test the network

In [ ]:
outputs = test(torch_model, test_loader)
print(outputs.shape)

In [9]:
del noisy_train_data
del train_data
del test_data
del train_loader

show the resutls

In [ ]:
from helpers.preprocessing import display
# Display the original and reconstructed images
torch_outputs =  np.transpose (outputs.cpu().numpy(), (0,2,1))

# Assuming you have a DataLoader named train_loader
data_iter = iter(test_loader)
first_batch = next(data_iter)

# Now first_batch contains the first batch of data
inputs = first_batch[0].float()  # Assuming float data type

inputs_tr = np.transpose(inputs, (0,2,1))
display(inputs_tr[:1], torch_outputs[:1])
print(torch_outputs[:1])

save the onnx file

In [11]:
save_onnx(torch_model, inputs[:1], '.', 'torch_original_model')

convert it to streaming


In [12]:
import onnx
# Load the ONNX model
model = onnx.load("torch_original_model.onnx")


In [ ]:
from streamease.onnx_streamer.streamer import StreamingConverter
streaming =  StreamingConverter(model, time_steps=1)

streaming.run()

streaming.print_info()
streaming.save_streaming_onnx('.', onnx_filename='torch_streaming_model')

Test the streaming model

In [ ]:
from utils.onnx_inference.streaming_inference import Inference

original_model ="torch_original_model.onnx"
streaming_model = "torch_streaming_model.onnx" 
s_test = Inference( streaming_model, time_steps=1, receptive_field=14, causal=True)

s_test.init_buffers()
input_data = inputs[:1].cpu().numpy()
# input_data = np.reshape(input_data, (1,32,28))

input_data = np.transpose(input_data, (0,2,1))
print(input_data.shape)
str_output = s_test.run(input_data, transpose=True)
display(input_data[:1],str_output[:1])

print(str_output[:1])

# NNTOOL

In [15]:
from nntool.api import NNGraph
from nntool.api.utils import model_settings, quantization_options, tensor_plot
import logging
# nntool_log = logging.getLogger('nntool')
# nntool_log.setLevel(logging.ERROR)

Now we load the graph

In [ ]:
model = NNGraph.load_graph('torch_original_model.onnx', use_onnx_names=True)
model.adjust_order()
model.fusions('scaled_match_group')
    
# Model show returns a table of information on the Graph
print(model.show())

# Model draw can open or save a PDF with a visual representation of the graph
# model.draw()


Let's run the non-streaming model in nntool

In [ ]:
data = inputs[:1].cpu().numpy()
output=model.execute(data)[-1]
print(output)

for item in output:
    display(data, item[:1])

Let's run the streaming model

In [ ]:
from utils.nntool_inference.streaming_inference import Inference

original_model ="torch_original_model.onnx"
streaming_model = "torch_streaming_model.onnx" 
s_model = NNGraph.load_graph(streaming_model, use_onnx_names=True)
# s_model.draw()
s_model.adjust_order()
# s_model.draw()
# The equivalent of the fusions --scale8 command. The fusions method can be given a series of fusions to apply
# fusions('name1', 'name2', etc)
s_model.fusions('scaled_match_group')

s_test = Inference(s_model, streaming_model, time_steps=1, receptive_field=14, causal=True)

s_test.init_buffers()
input_data = inputs[:1].cpu().numpy()
# input_data = np.reshape(input_data, (1,32,28))

input_tra = np.transpose(input_data, (0,2,1))
print(input_data[:1].shape)
nn_str_output = s_test.run(input_tra, i_transpose=False, o_transpose=False)
f_output = np.transpose(nn_str_output, (0,2,1))
display(input_data[:1],f_output)


In [ ]:
for i in range(len(nn_str_output)):
    print("#######################################")
    # print("golden model", torch_outputs[i])
    print("streaming model", nn_str_output[i])